### Imports

In [1]:
import json
import random
from datetime import datetime, timedelta

import os
import numpy as np
import re
from faker import Faker
from shapely.geometry import Point
import geopandas as gpd
from pyproj import Transformer

In [2]:
# Inicializa gerador de dados
fake = Faker('pt_BR')
fake2 = Faker('en_US')

# Configurações para localização
# Limites aproximados de latitude e longitude da cidade de São Paulo
LAT_RANGE = (-23.8, -23.4)
LON_RANGE = (-46.9, -46.3)

# Sistema de projeção plano (UTM Zona 23S, que cobre São Paulo)
transformer = Transformer.from_crs("EPSG:4326", "EPSG:31983", always_xy=True)

In [9]:
# 🔢 Gera um array de inteiros entre 0 e Z, com tamanho entre X e Y
def gerar_array_inteiros(x_min, y_max, z_max):
    tamanho = random.randint(x_min, y_max)
    return [random.randint(0, z_max) for _ in range(tamanho)]


# 🔢 Gera um array de floats entre 0 e Z, com tamanho entre X e Y
def gerar_array_floats(x_min, y_max, z_max):
    tamanho = random.randint(x_min, y_max)
    return [random.uniform(0, z_max) for _ in range(tamanho)]


# 🔢 Gera um inteiro entre 0 e Z
def gerar_inteiro(z_max):
    return random.randint(0, z_max)


# 🔢 Gera um float entre 0 e Z
def gerar_float(z_max):
    return random.uniform(0, z_max)


# 🔠 Gera um array de strings (palavras em inglês) com tamanho entre X e Y
def gerar_array_palavras(x_min, y_max):
    tamanho = random.randint(x_min, y_max)
    return [fake2.word() for _ in range(tamanho)]

In [27]:
# Função para gerar idade com distribuição normal truncada
def gerar_idade():
    idade = int(np.random.normal(loc=55, scale=20))  # Média 35 anos, desvio 20
    return max(0, min(120, idade))  # Trunca entre 0 e 100

# Função para gerar salário com distribuição exponencial
def gerar_salario():
    salario = np.random.exponential(scale=20000)  # Média aproximada 20000
    return round(min(salario, 100000), 2)  # Limita até 100000

# Função para gerar data de nascimento a partir da idade
def gerar_data_nascimento(idade):
    hoje = datetime.now()
    data_nascimento = hoje - timedelta(days=idade * 365 + random.randint(0, 364))
    return data_nascimento.isoformat() 
    
    #.strftime('%Y-%m-%d %H:%M:%S')

# Função para gerar ponto geográfico em São Paulo
def gerar_localizacao():
    lat = random.uniform(*LAT_RANGE)
    lon = random.uniform(*LON_RANGE)
    return lon, lat

# Função para gerar descrição aleatória
def gerar_descricao():
    return fake2.paragraph(nb_sentences=2, variable_nb_sentences=True)[:500]
    #return fake.text(max_nb_chars=100)

In [8]:
def gerar_lista_telefones(x_min, y_max):
    telefones = []
    tamanho = random.randint(x_min, y_max)
    for _ in range(tamanho):
        telefone = fake.phone_number()
        numero_limpo = int(re.sub(r'\D', '', telefone))
        telefones.append(numero_limpo)
    return telefones

def gerar_lista_precos(x_min, y_max):
    tamanho = random.randint(x_min, y_max)
    precos = [round(random.uniform(1, 100000), 2) for _ in range(tamanho)]
    return precos

In [6]:
def gerar_descricao():
    # Gera uma descrição de até 100 caracteres com frases curtas em português
    descricao = ''
    while len(descricao) < 300:
        frase = fake2.sentence()
        if len(descricao) + len(frase) + 1 > 300:
            break
        descricao += frase + ' '
    return descricao.strip()

In [7]:
# Função principal para gerar um registro
def gerar_registro2(qtde_data: int, limite: float):
    nome = fake.name()
    idade = gerar_inteiro(qtde_data)
    idade2 = gerar_inteiro(qtde_data)
    salario = gerar_float(limite)
    salario2 = gerar_float(limite)
    data_nascimento = gerar_data_nascimento(idade)
    array_int = gerar_array_inteiros(1, 15, qtde_data)
    array_float = gerar_array_floats(1, 15, limite)
    array_str = gerar_array_palavras(1, 15)

    lon, lat = gerar_localizacao()
    x, y = transformer.transform(lon, lat)  # Conversão para plano

    registro = {
        "nome": nome,
        "idade": idade,
        "data_nascimento": {"$date": data_nascimento},
        "salario": salario,
        "descricao": gerar_descricao(),
        "localizacao_casa": {
            "type": "Point",
            "coordinates": [lon, lat]
        },
        "localizacao_casa_plano": {
            "x": x,
            "y": y
        },
        "salario2": salario2,
        "idade2": idade2,
        "array_int": array_int,
        "array_float": array_float,
        "array_str": array_str,
    }
    return registro

In [32]:
# Função principal para gerar um registro
def gerar_registro():
    nome = fake.name()
    idade = gerar_idade()
    #idade2 = gerar_idade()
    salario = gerar_salario()
    salario2 = gerar_salario()
    data_nascimento = gerar_data_nascimento(idade)
    array_int = gerar_lista_telefones(0, 15)
    array_float = gerar_lista_precos(0, 15)
    lon, lat = gerar_localizacao()
    x, y = transformer.transform(lon, lat)  # Conversão para plano

    registro = {
        "name": nome,
        "age": idade,
        "birth_date": {"$date": data_nascimento},
        "wage": salario,
        "description": gerar_descricao(),
        "house_location_sphere": {
            "type": "Point",
            "coordinates": [lon, lat]
        },
        "house_location_plane": {
            "x": x,
            "y": y
        },
        "old_wage": salario2,
        "cellphones": array_int,
        "products_prices": array_float,        
    }
    return registro

def gerar_dados_sinteticos(qtd=10**7, pasta='./data/'):
    os.makedirs(pasta, exist_ok=True)

    for i in range(qtd):
        registro = gerar_registro()
        caminho_arquivo = os.path.join(pasta, f'registro_{i}.json')
        with open(caminho_arquivo, 'w', encoding='utf-8') as f:
            json.dump(registro, f, ensure_ascii=False, indent=4)

    print(f'{qtd} registros gerados na pasta "{pasta}"')

def gerar_dados_sinteticos_pior_caso(qtd=100000, limite=100000, pasta='./data/'):
    os.makedirs(pasta, exist_ok=True)

    for i in range(qtd):
        registro = gerar_registro2(qtd, limite)
        caminho_arquivo = os.path.join(pasta, f'registro_{i}.json')
        with open(caminho_arquivo, 'w', encoding='utf-8') as f:
            json.dump(registro, f, ensure_ascii=False, indent=4)

    print(f'{qtd} registros gerados na pasta "{pasta}"')

In [31]:
gerar_registro()

{'name': 'Luigi Farias',
 'age': 76,
 'birth_date': {'$date': '1949-02-26T11:19:09.956102'},
 'wage': 22916.7,
 'description': 'Stop huge character pick perform must long. Cost stock wide over.',
 'house_location_sphere': {'type': 'Point',
  'coordinates': [-46.39233893579017, -23.504732479308235]},
 'house_location_plane': {'x': 357837.4158349442, 'y': 7399915.7092201635},
 'old_wage': 41662.56,
 'cellphones': [5507115414294,
  5501119084397,
  7105466207,
  555141113743,
  5502144136870,
  555130464570,
  2188331048,
  5506133004730,
  1184826238,
  555161862109,
  1195326857,
  5506123914769,
  7196274064,
  5144870962],
 'products_prices': [50118.26,
  83104.9,
  96969.51,
  963.79,
  70783.41,
  79344.23,
  2629.81,
  16807.01,
  13010.56,
  10628.8,
  49522.96,
  93449.46,
  65290.62]}

In [10]:
gerar_dados_sinteticos()

100000 registros gerados na pasta "./data/"


In [11]:
import os
import json
import time
from pymongo import MongoClient, ASCENDING, DESCENDING, GEOSPHERE
from datetime import datetime

In [12]:
# 🔗 Conexão com o MongoDB (ajuste a URI conforme sua configuração)
client = MongoClient("mongodb://localhost:27017/")

# 🗄️ Criação do banco de dados e coleção
db = client["bd_mac5861"]
collection = db["collection"]

In [13]:
# 📥 Função para carregar arquivos JSON até um limite especificado
def carregar_dados(pasta="dados", limite=100000):
    arquivos = sorted([
        arq for arq in os.listdir(pasta) if arq.startswith("registro_") and arq.endswith(".json")
    ])
    arquivos = arquivos[:limite]

    documentos = []
    for arq in arquivos:
        caminho = os.path.join(pasta, arq)
        with open(caminho, 'r', encoding='utf-8') as f:
            dado = json.load(f)

            # Conversão do campo de data (MongoDB aceita datetime)
            if "data_nascimento" in dado and "$date" in dado["data_nascimento"]:
                dado["data_nascimento"] = datetime.fromisoformat(dado["data_nascimento"]["$date"])

            documentos.append(dado)

    if documentos:
        collection.insert_many(documentos)
        print(f"Inseridos {len(documentos)} documentos.")
    else:
        print("Nenhum documento foi carregado.")

In [14]:
# 🔧 Criação dos índices
def criar_indices():
    # Índices simples
    collection.create_index([("nome", ASCENDING)])
    collection.create_index([("idade", ASCENDING)])
    collection.create_index([("idade2", ASCENDING)])
    collection.create_index([("salario", ASCENDING)])
    collection.create_index([("salario2", ASCENDING)])
    collection.create_index([("data_nascimento", ASCENDING)])
    collection.create_index([("descricao", ASCENDING)])
    collection.create_index([("localizacao_casa", GEOSPHERE)])  # Índice geoespacial esférico
    collection.create_index([("localizacao_casa_plano.x", ASCENDING), ("localizacao_casa_plano.y", ASCENDING)])

    # Índices compostos
    collection.create_index([("idade", ASCENDING), ("salario", DESCENDING)])
    collection.create_index([("idade", ASCENDING), ("data_nascimento", ASCENDING)])
    collection.create_index([("salario", DESCENDING), ("descricao", ASCENDING)])

    print("Índices criados.")

In [15]:
# 🔍 Consultas exemplo
def consultas_exemplo():
    print("\nExemplos de consultas:")

    # Consulta 1: Pessoas com idade entre 30 e 50
    resultado = collection.find({"idade": {"$gte": 30, "$lte": 50}})
    print(f"Idades entre 30 e 50: {resultado.count()} documentos")

    # Consulta 2: Pessoas com salário acima de 50 mil
    resultado = collection.find({"salario": {"$gt": 50000}})
    print(f"Salário acima de 50k: {resultado.count()} documentos")

    # Consulta 3: Buscar por nome específico
    resultado = collection.find({"nome": "Erick Costela"})
    for doc in resultado:
        print(doc)

    # Consulta 4: Consulta geoespacial - próximo de uma coordenada
    ponto_referencia = {"type": "Point", "coordinates": [-46.633309, -23.55052]}  # Centro de SP
    resultado = collection.find({
        "localizacao_casa": {
            "$near": {
                "$geometry": ponto_referencia,
                "$maxDistance": 5000  # 5km
            }
        }
    })
    print(f"Documentos próximos ao centro de SP (5km): {resultado.count()} documentos")

In [17]:
# ⏱️ Medição do tempo de resposta das consultas
def medir_tempo_consulta(consulta):
    inicio = time.time()
    resultado = list(consulta)
    fim = time.time()
    tempo = fim - inicio
    print(f"Tempo de resposta: {tempo:.6f} segundos")
    return resultado


# 🚀 Executando tudo
if __name__ == "__main__":
    # 1. Limpa a coleção (opcional para rodar do zero)
    collection.delete_many({})
    print("Coleção limpa.")

    # 2. Carregar dados
    carregar_dados(pasta="data", limite=100000)

    # 3. Criar índices
    criar_indices()

    # 4. Consultas e medição de tempo

    print("\nMedindo tempos de consultas:")

    # Consulta 1: Idade entre 30 e 50
    consulta1 = collection.find({"idade": {"$gte": 30, "$lte": 50}})
    medir_tempo_consulta(consulta1)

    # Consulta 2: Salário acima de 50k
    consulta2 = collection.find({"salario": {"$gt": 50000}})
    medir_tempo_consulta(consulta2)

    # Consulta 3: Geoespacial - próximos ao centro de SP
    ponto = {"type": "Point", "coordinates": [-46.633309, -23.55052]}
    consulta3 = collection.find({
        "localizacao_casa": {
            "$near": {
                "$geometry": ponto,
                "$maxDistance": 5000
            }
        }
    })
    medir_tempo_consulta(consulta3)

Coleção limpa.
Inseridos 100000 documentos.
Índices criados.

Medindo tempos de consultas:
Tempo de resposta: 0.568578 segundos
Tempo de resposta: 0.071140 segundos
Tempo de resposta: 0.038369 segundos


In [18]:
consulta1 = collection.find({"idade": {"$gte": 30, "$lte": 50}})
print(consulta1)

In [27]:
consulta1 = collection.find(
    {"idade": {"$gte": 30, "$lte": 50}}
).explain()
from pprint import pprint
pprint(consulta1)

{'command': {'$db': 'bd_mac5861',
             'filter': {'idade': {'$gte': 30, '$lte': 50}},
             'find': 'collection'},
 'executionStats': {'allPlansExecution': [{'executionStages': {'advanced': 101,
                                                               'alreadyHasObj': 0,
                                                               'docsExamined': 101,
                                                               'executionTimeMillisEstimate': 0,
                                                               'inputStage': {'advanced': 101,
                                                                              'direction': 'forward',
                                                                              'dupsDropped': 0,
                                                                              'dupsTested': 0,
                                                                              'executionTimeMillisEstimate': 0,
                          

In [28]:
pprint(consulta1['executionStats'])

{'allPlansExecution': [{'executionStages': {'advanced': 101,
                                            'alreadyHasObj': 0,
                                            'docsExamined': 101,
                                            'executionTimeMillisEstimate': 0,
                                            'inputStage': {'advanced': 101,
                                                           'direction': 'forward',
                                                           'dupsDropped': 0,
                                                           'dupsTested': 0,
                                                           'executionTimeMillisEstimate': 0,
                                                           'indexBounds': {'idade': ['[30, '
                                                                                     '50]']},
                                                           'indexName': 'idade_1',
                                                         

In [33]:
stats = db.command("collStats", "collection")

print(stats)

print("\n📊 Tamanho de cada índice:")
for indice, tamanho in stats['indexSizes'].items():
    print(f" - {indice}: {tamanho / 1024:.2f} KB")
print(f"Index Size: {stats['totalIndexSize']/1024/1024:.2f} MB")
print(f"Storage Size: {stats['storageSize']/1024/1024:.2f} MB")

{'ns': 'bd_mac5861.collection', 'size': 86256374, 'count': 100000, 'avgObjSize': 862, 'numOrphanDocs': 0, 'storageSize': 58540032, 'freeStorageSize': 0, 'capped': False, 'wiredTiger': {'metadata': {'formatVersion': 1}, 'creationString': 'access_pattern_hint=none,allocation_size=4KB,app_metadata=(formatVersion=1),assert=(commit_timestamp=none,durable_timestamp=none,read_timestamp=none,write_timestamp=off),block_allocation=best,block_compressor=snappy,cache_resident=false,checksum=on,colgroups=,collator=,columns=,dictionary=0,encryption=(keyid=,name=),exclusive=false,extractor=,format=btree,huffman_key=,huffman_value=,ignore_in_memory_cache_size=false,immutable=false,import=(compare_timestamp=oldest_timestamp,enabled=false,file_metadata=,metadata_file=,panic_corrupt=true,repair=false),internal_item_max=0,internal_key_max=0,internal_key_truncate=true,internal_page_max=4KB,key_format=q,key_gap=10,leaf_item_max=0,leaf_key_max=0,leaf_page_max=32KB,leaf_value_max=64MB,log=(enabled=true),lsm=(

In [29]:
stats = db["collection"].stats()
print(f"Storage Size: {stats['IndexSize']/1024/1024:.2f} MB")
print(f"Index Size: {stats['totalIndexSize']/1024/1024:.2f} MB")
print(f"Storage Size: {stats['storageSize']/1024/1024:.2f} MB")


TypeError: 'Collection' object is not callable. If you meant to call the 'stats' method on a 'Collection' object it is failing because no such method exists.